In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/mnt/software/'
import sys
import gc
# assuming data, models, engine in flicc directory:
flicc_path = os.path.join(os.path.dirname(os.getcwd()), '')
sys.path.append(flicc_path)
import torch
from data import ClimateDataset
from models import ClassificationModel
from engine import Engine

# import warnings
# warnings.filterwarnings("ignore")

In [2]:
best_config = {'bert-base-uncased':{'lr':5.0e-5},
                'roberta-large':{'lr':5.0e-5},
                'gpt2':{'lr':5.0e-5},
                'bigscience/bloom-560m':{'lr':5.0e-5},
                'facebook/opt-350m':{'lr':1.0e-5},
                'EleutherAI/gpt-neo-1.3B':{'lr':5.0e-5}, 
                'microsoft/deberta-base':{'lr':1.0e-5},
                'microsoft/deberta-v2-xlarge':{'lr':1.0e-5}}

In [3]:
results = {'test_acc':[],
           'test_f1':[],
           'eval_acc':[],
           'eval_f1':[],
           'g':[]}

In [4]:
gamma = [2,4,8,16]

for model_checkpoint in best_config.keys():
    for g in gamma:
        print(f'Grid search {model_checkpoint}, learning rate {best_config[model_checkpoint]["lr"]}')
        data = ClimateDataset(model_to_train=4,model_checkpoint=model_checkpoint,dataset_url=flicc_path,batch_size=32)
        data.setup_dataloaders()
        model = ClassificationModel(model_checkpoint=data.model_checkpoint,num_labels=data.num_labels)
        trainer = Engine(epochs=30,labels=data.labels)
        trainer.model = model.model
        trainer.dataset_encoded = data.dataset_encoded
        test_acc, test_f1, eval_acc, eval_f1 = trainer.run(lr=best_config[model_checkpoint]['lr'],
                                                            wd=0.0,
                                                            train_dataloader=data.train_dataloader,
                                                            eval_dataloader=data.eval_dataloader,
                                                            test_dataloader=data.test_dataloader,
                                                            focalloss=True,
                                                            gamma=g,
                                                            early_stop=3)
        results['test_acc'].append(test_acc)
        results['test_f1'].append(test_f1)
        results['eval_acc'].append(eval_acc)
        results['eval_f1'].append(eval_f1)
        results['g'].append(g)
        print('### '*10)
        print(results)
        print('### '*10)
        with torch.no_grad():
            torch.cuda.empty_cache()
        del data, model, trainer
        gc.collect()

Grid search bert-base-uncased, learning rate 5e-05


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


1 / 30: Train Loss:	21.5878	Val Loss:	1.8216	Accuracy:	0.4026	F1:	0.3431 *
2 / 30: Train Loss:	12.3960	Val Loss:	1.6948	Accuracy:	0.4530	F1:	0.4332 *
3 / 30: Train Loss:	6.2709	Val Loss:	1.4211	Accuracy:	0.5252	F1:	0.5152 *
4 / 30: Train Loss:	2.7156	Val Loss:	1.2350	Accuracy:	0.6018	F1:	0.5858 *
5 / 30: Train Loss:	1.1768	Val Loss:	1.2149	Accuracy:	0.6368	F1:	0.6224 *
6 / 30: Train Loss:	0.5279	Val Loss:	1.1698	Accuracy:	0.6543	F1:	0.6415 *
7 / 30: Train Loss:	0.2480	Val Loss:	1.1532	Accuracy:	0.6630	F1:	0.6359
8 / 30: Train Loss:	0.1090	Val Loss:	1.1358	Accuracy:	0.6586	F1:	0.6308
9 / 30: Train Loss:	0.0473	Val Loss:	1.1311	Accuracy:	0.6630	F1:	0.6427 *
10 / 30: Train Loss:	0.0343	Val Loss:	1.1182	Accuracy:	0.6783	F1:	0.6574 *
11 / 30: Train Loss:	0.0320	Val Loss:	1.1245	Accuracy:	0.6696	F1:	0.6398
12 / 30: Train Loss:	0.0250	Val Loss:	1.1340	Accuracy:	0.6761	F1:	0.6510
13 / 30: Train Loss:	0.0174	Val Loss:	1.1267	Accuracy:	0.6761	F1:	0.6507
No improvement for 3 epochs. Stopping earl

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


1 / 30: Train Loss:	17.7481	Val Loss:	1.8447	Accuracy:	0.4464	F1:	0.3860 *
2 / 30: Train Loss:	9.3680	Val Loss:	1.6075	Accuracy:	0.4989	F1:	0.4667 *
3 / 30: Train Loss:	4.2528	Val Loss:	1.4378	Accuracy:	0.5295	F1:	0.5149 *
4 / 30: Train Loss:	1.8639	Val Loss:	1.3205	Accuracy:	0.5558	F1:	0.5455 *
5 / 30: Train Loss:	0.8342	Val Loss:	1.2410	Accuracy:	0.6236	F1:	0.6083 *
6 / 30: Train Loss:	0.3459	Val Loss:	1.1654	Accuracy:	0.6346	F1:	0.6152 *
7 / 30: Train Loss:	0.1744	Val Loss:	1.1782	Accuracy:	0.6214	F1:	0.5916
8 / 30: Train Loss:	0.1141	Val Loss:	1.1194	Accuracy:	0.6521	F1:	0.6132
9 / 30: Train Loss:	0.0407	Val Loss:	1.1247	Accuracy:	0.6477	F1:	0.6089
No improvement for 3 epochs. Stopping early.
best (higgest macro f1-score) val results:
                         precision    recall  f1-score   support

             ad hominem       0.74      0.58      0.65        67
               anecdote       0.85      0.81      0.83        43
         cherry picking       0.74      0.57      0.65 

...

1 / 30: Train Loss:	21.0395	Val Loss:	2.1011	Accuracy:	0.2867	F1:	0.1971 *
2 / 30: Train Loss:	10.1067	Val Loss:	1.2699	Accuracy:	0.6171	F1:	0.6000 *
3 / 30: Train Loss:	4.0454	Val Loss:	1.0741	Accuracy:	0.6389	F1:	0.6283 *
4 / 30: Train Loss:	1.9878	Val Loss:	0.9700	Accuracy:	0.6652	F1:	0.6700 *
5 / 30: Train Loss:	1.1254	Val Loss:	0.9061	Accuracy:	0.7068	F1:	0.7002 *
6 / 30: Train Loss:	0.4867	Val Loss:	0.8808	Accuracy:	0.7155	F1:	0.7219 *
7 / 30: Train Loss:	0.2945	Val Loss:	0.8893	Accuracy:	0.7090	F1:	0.7130
8 / 30: Train Loss:	0.2070	Val Loss:	0.9257	Accuracy:	0.7002	F1:	0.7013
9 / 30: Train Loss:	0.2254	Val Loss:	0.9679	Accuracy:	0.7002	F1:	0.6907
No improvement for 3 epochs. Stopping early.
best (higgest macro f1-score) val results:
                         precision    recall  f1-score   support

             ad hominem       0.81      0.63      0.71        67
               anecdote       0.97      0.84      0.90        43
         cherry picking       0.63      0.71      0.67

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v2-xlarge and are newly initialized: ['classifier.weight', 'pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


1 / 30: Train Loss:	11.8173	Val Loss:	1.5854	Accuracy:	0.5580	F1:	0.5152 *
2 / 30: Train Loss:	5.0966	Val Loss:	1.6952	Accuracy:	0.5317	F1:	0.5115
3 / 30: Train Loss:	2.7530	Val Loss:	1.1896	Accuracy:	0.6630	F1:	0.6555 *
4 / 30: Train Loss:	1.8121	Val Loss:	1.1379	Accuracy:	0.6696	F1:	0.6509
5 / 30: Train Loss:	2.8761	Val Loss:	1.1428	Accuracy:	0.6608	F1:	0.6301
6 / 30: Train Loss:	2.8735	Val Loss:	1.1391	Accuracy:	0.6608	F1:	0.6444
No improvement for 3 epochs. Stopping early.
best (higgest macro f1-score) val results:
                         precision    recall  f1-score   support

             ad hominem       0.76      0.67      0.71        67
               anecdote       1.00      0.81      0.90        43
         cherry picking       0.71      0.64      0.67        56
      conspiracy theory       0.69      0.87      0.77        39
           fake experts       0.56      0.75      0.64        12
           false choice       0.60      0.69      0.64        13
      false equival

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v2-xlarge and are newly initialized: ['classifier.weight', 'pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


1 / 30: Train Loss:	6.3958	Val Loss:	1.7630	Accuracy:	0.5252	F1:	0.4835 *
2 / 30: Train Loss:	2.1492	Val Loss:	1.5539	Accuracy:	0.5930	F1:	0.5705 *
3 / 30: Train Loss:	0.8571	Val Loss:	1.3015	Accuracy:	0.6674	F1:	0.6626 *
4 / 30: Train Loss:	0.4105	Val Loss:	1.2486	Accuracy:	0.6740	F1:	0.6725 *
5 / 30: Train Loss:	0.2148	Val Loss:	1.1620	Accuracy:	0.7112	F1:	0.7002 *
6 / 30: Train Loss:	0.0963	Val Loss:	1.1113	Accuracy:	0.6980	F1:	0.6820
7 / 30: Train Loss:	0.0507	Val Loss:	1.0751	Accuracy:	0.7155	F1:	0.6975
8 / 30: Train Loss:	0.0248	Val Loss:	1.0321	Accuracy:	0.7243	F1:	0.7067 *
9 / 30: Train Loss:	0.0324	Val Loss:	1.0372	Accuracy:	0.6980	F1:	0.6916
10 / 30: Train Loss:	0.0139	Val Loss:	1.0196	Accuracy:	0.7243	F1:	0.7129 *
11 / 30: Train Loss:	0.0136	Val Loss:	1.0242	Accuracy:	0.7265	F1:	0.7125
12 / 30: Train Loss:	0.0078	Val Loss:	1.0191	Accuracy:	0.7221	F1:	0.7123
13 / 30: Train Loss:	0.0079	Val Loss:	1.0079	Accuracy:	0.7199	F1:	0.7064
No improvement for 3 epochs. Stopping early.
b